# Skaffold
- [Home](https://skaffold.dev/)
- [skaffold.yaml](https://skaffold.dev/docs/references/yaml/)

Skaffold is a command line tool that facilitates continuous development for container based & Kubernetes applications. 

Skaffold handles the workflow for building, pushing, and deploying your application, and provides building blocks for creating CI/CD pipelines. 
This enables you to focus on iterating on your application locally while Skaffold continuously deploys to your local or remote Kubernetes cluster, local Docker environment or Cloud Run project.

# Workflow

Every time you run skaffold dev, the system

1. Collects and watches your source code for changes
2. Syncs files directly to pods if user marks them as syncable
3. Builds artifacts from the source code
4. Tests the built artifacts using container-structure-tests
5. Tags the artifacts
6. Pushes the artifacts
7. Deploys the artifacts
8. Monitors the deployed artifacts
9. Cleans up deployed artifacts on exit (Ctrl+C).

| Skaffold Pipeline stages | Description                                                      |
| :----------------------- | :--------------------------------------------------------------- |
| Init                     | generate a starting point for Skaffold configuration             |
| Build                    | build images with different builders                             |
| Tag                      | tag images based on different policies                           |
| Test                     | test images with structure tests                                 |
| Deploy                   | deploy with kubectl, kustomize or helm                           |
| File Sync                | sync changed files directly to containers                        |
| Log Tailing              | tail logs from workloads                                         |
| Port Forwarding          | forward ports from services and arbitrary resources to localhost |
| Cleanup                  | cleanup manifests and images                                     |


# CLI

In [3]:
!skaffold build --help

Build, test and tag the artifacts

Examples:
  # Build all the artifacts
  skaffold build

  # Build artifacts with a profile activated
  skaffold build -p <profile>

  # Build artifacts whose image name contains <db>
  skaffold build -b <db>

  # Quietly build artifacts and output the image names as json
  skaffold build -q > build_result.json

  # Build the artifacts and then deploy them
  skaffold build -q | skaffold deploy --build-artifacts -

  # Print the final image names
  skaffold build -q --dry-run

Options:
      --assume-yes=false: If true, skaffold will skip yes/no confirmation from the user and default to yes
      --build-concurrency=-1: Number of concurrently running builds. Set to 0 to run all builds in parallel. Doesn't violate build order among dependencies.
  -b, --build-image=[]: Only build artifacts with image names that contain the given substring. Default is to build sources for all artifacts
      --cache-artifacts=true: Set to false to disable default caching 

In [ ]:
# !skaffold build -p <profile> 
# !skaffold dev -p <profile> --tail

# Deploy to Kubernetes

Tools:
- Jib
- Helm
- IDEA plugin: Cloud Code: Kubernetes

## Jib

```xml
<plugin>
    <groupId>com.google.cloud.tools</groupId>
    <artifactId>jib-maven-plugin</artifactId>
    <configuration>
        <containerizingMode>packaged</containerizingMode>
        <from>
            <image>${jib.image.from}</image>
        </from>
        <to>
            <image>${jib.image.to}</image>
            <auth>
                <username>${harbor.username}</username>
                <password>${harbor.password}</password>
            </auth>
        </to>
        <container>
            <jvmFlags>
                <jvmFlag>-Xmx512m</jvmFlag>
            </jvmFlags>
            <environment>
                <TZ>Asia/Shanghai</TZ>
            </environment>
            <volumes>
                <volume>/tmp</volume>
            </volumes>
            <ports>
                <port>80</port>
            </ports>
            <mainClass>xxx</mainClass>
            <format>OCI</format>
        </container>
        <allowInsecureRegistries>true</allowInsecureRegistries>
    </configuration>
</plugin>
```

## skaffold.yaml

```yaml
apiVersion: skaffold/v2beta29
kind: Config
metadata:
  name: app-platform

profiles:
  - name: biz-core-app
    build:
      artifacts:
        - image: biz-core-app # IP/app-platform/biz-core-app
          jib:
            project: biz-core-app
            args:
              - -Dmaven.test.skip=true
    #      local: # DEV_ENV
    #        push: true
    deploy:
      helm:
        releases:
          - name: biz-core-org
            namespace: app-platform
            chartPath: biz-core-app/Charts
            valuesFiles:
              - biz-core-app/Charts/values.yaml
            setValues:
              image.repository: biz-core-app
              image.pullPolicy: Never # DEV_ENV
            version: 0.1.0
        flags:
          upgrade:
            - --cleanup-on-fail
            - --install
```

## Helm

values.yaml
```yaml
env:
  JAVA_VERSION: 17
  JAVA_TOOL_OPTIONS: -agentlib:jdwp=transport=dt_socket,server=y,address=5005,suspend=n,quiet=y

livenessProbe:
  httpGet:
    path: /actuator/health/liveness
    port: http
readinessProbe:
  httpGet:
    path: /actuator/health/readiness
    port: http
```
deployment.yaml
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: {{ include "biz-core-app.fullname" . }}
  labels:
    {{- include "biz-core-app.labels" . | nindent 4 }}
spec:
  {{- if not .Values.autoscaling.enabled }}
  replicas: {{ .Values.replicaCount }}
  {{- end }}
  selector:
    matchLabels:
      {{- include "biz-core-app.selectorLabels" . | nindent 6 }}
  template:
    metadata:
      {{- with .Values.podAnnotations }}
      annotations:
        {{- toYaml . | nindent 8 }}
      {{- end }}
      labels:
        {{- include "biz-core-app.labels" . | nindent 8 }}
        {{- with .Values.podLabels }}
        {{- toYaml . | nindent 8 }}
        {{- end }}
    spec:
      {{- with .Values.imagePullSecrets }}
      imagePullSecrets:
        {{- toYaml . | nindent 8 }}
      {{- end }}
      serviceAccountName: {{ include "biz-core-app.serviceAccountName" . }}
      securityContext:
        {{- toYaml .Values.podSecurityContext | nindent 8 }}
      containers:
        - name: {{ .Chart.Name }}
          securityContext:
            {{- toYaml .Values.securityContext | nindent 12 }}
          image: "{{ .Values.image.repository }}:{{ .Values.image.tag | default .Chart.AppVersion }}"
          imagePullPolicy: {{ .Values.image.pullPolicy }}
          env:
            {{- range $key, $val := .Values.env }}
            - name: {{ $key }}
              value: {{ $val | quote }}
            {{- end }}
          ports:
            - name: http
              containerPort: {{ .Values.service.port }}
              protocol: TCP
            - name: jdwp
              containerPort: 5005
              protocol: TCP
          livenessProbe:
            {{- toYaml .Values.livenessProbe | nindent 12 }}
          readinessProbe:
            {{- toYaml .Values.readinessProbe | nindent 12 }}
          resources:
            {{- toYaml .Values.resources | nindent 12 }}
          {{- with .Values.volumeMounts }}
          volumeMounts:
            {{- toYaml . | nindent 12 }}
          {{- end }}
      {{- with .Values.volumes }}
      volumes:
        {{- toYaml . | nindent 8 }}
      {{- end }}
      {{- with .Values.nodeSelector }}
      nodeSelector:
        {{- toYaml . | nindent 8 }}
      {{- end }}
      {{- with .Values.affinity }}
      affinity:
        {{- toYaml . | nindent 8 }}
      {{- end }}
      {{- with .Values.tolerations }}
      tolerations:
        {{- toYaml . | nindent 8 }}
      {{- end }}

```

# Deploy to Docker

Tools:
- Jib

## skaffold.yaml


```yaml
apiVersion: skaffold/v2beta29
kind: Config
metadata:
  name: app-platform

portForward:
  - resourceType: Container
    resourceName: app-platform_inf-base-msg
    address: 0.0.0.0
    port: 6002
    localPort: 6002
  - resourceType: Container
    resourceName: app-platform_inf-base-msg
    address: 0.0.0.0
    port: 5005
    localPort: 15005

profiles:
  - name: inf-base-msg
    build:
      artifacts:
        - image: app-platform_inf-base-msg
          jib:
            project: inf-base/inf-base-msg
            args:
              - -Dmaven.test.skip=true
              - -Djib.container.jvmFlags=-Xmx512m,-Xdebug,-Dspring.profiles.active=local,-agentlib:jdwp=transport=dt_socket\,server=y\,address=*:5005\,suspend=n\,quiet=y
              - -Djib.container.environment=MYSQL_HOST="192.168.0.122",MYSQL_PORT=3306,MYSQL_DATABASE="xxx",MYSQL_USERNAME="root",MYSQL_PASSWORD="xxx"
    deploy:
      docker:
        images:
          - app-platform_inf-base-msg
```

## skaffold.env

```shell
DOCKER_HOST=tcp://192.168.0.166:2375
```

## JDWP

```shell
-Djib.container.jvmFlags=-Xmx512m,-Xdebug,-Dspring.profiles.active=local,-agentlib:jdwp=transport=dt_socket\,server=y\,address=*:5005\,suspend=n\,quiet=y
```

## run

In [ ]:
!skaffold dev -p inf-base-msg